In [2]:
import alpaca_trade_api as alpaca
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import hvplot.pandas

# Replace YOUR_API_KEY and YOUR_API_SECRET with your Alpaca API key and secret
alpaca_api = alpaca.REST('PKU5LEIDZZV83Y5ENASI', 'nvzOwJpyvr73GTitTDyga0MkI2Qd6RsK4PmhItbu', api_version='v2')

# Set the ticker symbol and time frame
ticker = "SPY"
timeframe = "30Min"

# Set the start and end dates for the data
start_date = "2012-01-01T00:00:00-00:00"
end_date = "2023-01-01T00:00:00-00:00"

# Retrieve the data
data = alpaca_api.get_bars(ticker,  timeframe, start_date, end_date).df

# Convert the data to a Pandas dataframe
df = pd.DataFrame(data)

# Calculate the 3 period moving average of the close

#df=df.drop(columns=['open','high','low','volume','trade_count','vwap'])
           
df["Signal"] = 0.0
#df['HH5']=df['high'].rolling(30).max()
#df['LL5']=df['low'].rolling(30).min()
df['SMA30']=df['close'].rolling(30).mean()

df['closePct']=df['close'].pct_change()-df['SMA30'].pct_change()
df['Tstat30']=(df['close']-df['SMA30'])/df['close'].rolling(30).std()



           
df


,close,high,low,trade_count,open,volume,vwap,Signal,SMA30,closePct,Tstat30
timestamp,,,,,,,,,,,
2016-01-01 00:00:00+00:00,204.01,204.090,204.01,11,204.05,1600,204.036606,0.0,NaN,NaN,NaN
2016-01-01 00:30:00+00:00,204.07,204.090,204.02,14,204.04,4057,204.063591,0.0,NaN,NaN,NaN
2016-01-04 09:00:00+00:00,201.38,201.390,200.83,108,201.18,23952,201.014637,0.0,NaN,NaN,NaN
2016-01-04 09:30:00+00:00,200.84,201.390,200.84,63,201.36,17794,201.077787,0.0,NaN,NaN,NaN
2016-01-04 10:00:00+00:00,201.04,201.130,200.86,53,200.87,40158,200.985194,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 21:00:00+00:00,382.99,383.130,382.31,6877,382.44,14611402,382.527370,0.0,381.224757,0.001443,1.465088
2022-12-30 21:30:00+00:00,383.04,383.060,382.43,1538,382.99,386776,382.971375,0.0,381.222757,0.000136,1.512231
2022-12-30 22:00:00+00:00,383.08,383.089,382.95,305,383.04,90859,383.014445,0.0,381.232423,0.000079,1.519611


In [3]:
df

dfCopy=df.copy()

dfCopy=dfCopy.reset_index()



lookahead=5

dfCopy['signal']=0

UpsideBreakout=False
DownsideBreakout=False

for index, row in dfCopy.iterrows():
    
    #print(lookahead)
    
    '''if(index<=dfCopy['close'].idxmax()-lookahead):
        print(dfCopy['close'].idxmax()-lookahead)
        break'''
    '''if (index<index-lookahead ):
        
        print(index-lookahead)'''
    
    HH=dfCopy.loc[index,'vwap']
    LL=dfCopy.loc[index,'vwap']
    if(index<dfCopy['close'].idxmax()-lookahead):
        for x in range(lookahead):
            
            if x==0:
                y=0
            else:
                y=x-1



            if(dfCopy.loc[index+x, 'close']>dfCopy.loc[index+x, 'SMA30'] and dfCopy.loc[index+y, 'close']<dfCopy.loc[index+y, 'SMA30']):
                UpsideBreakout=True
                break
                #dfCopy.loc[index+x, 'signal']=1

            if(dfCopy.loc[index+x, 'close']<dfCopy.loc[index+x, 'SMA30'] and dfCopy.loc[index+y, 'close']>dfCopy.loc[index+y, 'SMA30']):
                DownsideBreakout=True
                break
                #dfCopy.loc[index+x, 'signal']=-1

    if UpsideBreakout==True:
        for x in range(lookahead):
            dfCopy.loc[index+x, 'signal']=1
        UpsideBreakout=False
        
    if DownsideBreakout==True:
        for x in range(lookahead):
            dfCopy.loc[index+x, 'signal']=-1
        DownsideBreakout=False
            

                
print(index)


display(dfCopy['signal'].value_counts())
dfCopy.to_csv('DfCopy.csv')
dfCopy.set_index('timestamp', inplace=True)

54938


signal
 0    35344
 1     9920
-1     9675
Name: count, dtype: int64

In [4]:
dfCopy=dfCopy.dropna()

y = dfCopy['signal'].shift(-1)

# Review the value counts
y.value_counts()

signal
 0.0    35314
 1.0     9920
-1.0     9674
Name: count, dtype: int64

In [5]:
# Select the start of the training period

from datetime import datetime
from pandas.tseries.offsets import DateOffset
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

X = dfCopy[['closePct','Tstat30']].dropna()

X.to_csv('X.csv')


training_begin = X.index.min()

# Display the training begin date
print(training_begin)

training_end = X.index.min() + DateOffset(months=60)

# Display the training end date
print(training_end)

# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
display(X_train.head())
display(X_train.tail())

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_test.head()

# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

2016-01-04 23:00:00+00:00
2021-01-04 23:00:00+00:00


,closePct,Tstat30
timestamp,,
2016-01-04 23:00:00+00:00,0.001623,1.148341
2016-01-04 23:30:00+00:00,0.000344,1.530774
2016-01-05 00:00:00+00:00,0.000234,1.555045
2016-01-05 00:30:00+00:00,-0.000488,1.391844
2016-01-05 09:00:00+00:00,-0.006290,-0.080014


,closePct,Tstat30
timestamp,,
2021-01-04 21:00:00+00:00,-0.000114,-0.988663
2021-01-04 21:30:00+00:00,0.001776,-0.809569
2021-01-04 22:00:00+00:00,0.000159,-0.789603
2021-01-04 22:30:00+00:00,0.000486,-0.740283
2021-01-04 23:00:00+00:00,-0.001166,-0.847926


In [6]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC(probability=True) 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred

# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)


C:\Users\nihar\anaconda3\Lib\site-packages\sklearn\utils\multiclass.py:380: RuntimeWarning: invalid value encountered in cast
  if xp.any(data != data.astype(int)):


ValueError: Input y_true contains NaN.

In [ ]:
4991/9151

In [11]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import numpy as np


y_probs = svm_model.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)

# Plot the precision-recall curve
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

# Find the threshold with the highest F1 score
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold_index = np.argmax(precision)
best_threshold = thresholds[best_threshold_index]

# Make predictions with the best threshold
y_pred = (y_probs <= best_threshold).astype(int)

C:\Users\nihar\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
C:\Users\nihar\anaconda3\Lib\site-packages\sklearn\utils\multiclass.py:380: RuntimeWarning: invalid value encountered in cast
  if xp.any(data != data.astype(int)):


ValueError: Input y_true contains NaN.

In [ ]:
print(best_threshold)

print(np.argmax(precision))

svm_testing_report = classification_report(y_test, y_pred)

# Print the classification report
print(svm_testing_report)

In [ ]:
X_test

In [10]:

for x in range(1,100,1):
    threshold=float(x)/100
    print(threshold)
    y_pred = (y_probs >= threshold).astype(int)
    svm_testing_report = classification_report(y_test, y_pred)

    # Print the classification report
    print(svm_testing_report)
    
    

0.01


NameError: name 'y_probs' is not defined

In [9]:

import matplotlib.pyplot as plt

thresholds = []
precisions = []
recalls = []
f1_scores = []

# Iterate over thresholds from 0.01 to 0.99
for x in range(1, 100, 1):
    threshold = float(x) / 100
    thresholds.append(threshold)
    
    # Make predictions based on the current threshold
    y_pred = (y_probs >= threshold).astype(int)
    
    # Calculate precision, recall, and F1-score
    precision = precision_score(y_test_binary, y_pred, zero_division=0)
    recall = recall_score(y_test_binary, y_pred, zero_division=0)
    f1 = f1_score(y_test_binary, y_pred, zero_division=0)
    
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

# Plot precision, recall, and F1-score
plt.figure(figsize=(10, 6))
plt.plot(thresholds, precisions, label='Precision')
plt.plot(thresholds, recalls, label='Recall')
plt.plot(thresholds, f1_scores, label='F1 Score')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision, Recall, and F1 Score vs. Threshold')
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'model' is not defined